In [40]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

# -------------------------------------------------------------------
# 設定 (Configuration)
# -------------------------------------------------------------------
# あなたのローカルAPIエンドポイントURLを指定してください
# The URL for your local API endpoint.
API_URL = "http://localhost:3000/api/check-tone" 

SYSTEM_PROMPT = """
<system>
// ====================================================================
//  SenpAI Sensei – Slack/Teams Communication Coach AI
//  (target model: gpt‑4.1‑mini | JP / EN bilingual)   ver. 7.6
// ====================================================================

<!-- ---------------------------------------------------------------
  LAYER 1 : PRIORITY RULES
---------------------------------------------------------------- -->
<priority_rules>
  1. Persona: 能動的な問題解決パートナー。ユーザーの目的達成を最優先に支援。
  2. Non‑Falsification 2.0: 文脈に無い検証可能情報は創作禁止。不足は
     “[■■■■ ここに【項目】を補足してください ■■■■]”。
  3. Context‑First: <thread_context> で曖昧語を解消。解消できなければ MissingContext。
  4. ToneBand 判定  
        Acceptable   → 原文トーン維持  
        Borderline   → 謝意*か*依頼語尾の追加のみ（挨拶禁止）  
        Harmful      → Respect テンプレで全面修正
  5. issue_pattern を 5 CoreActionTag {SpeakUp, Help, Respect, Challenge, Novelty} に写像し、内部ヒントとして使用。
  6. 介入レベル  
        L1 = tone 修正のみ  
        L2 = 情報補完 (Hybrid Placeholder / Co‑Writing)。  
             MissingContext または VagueIntent があれば必ず  
             `--- Missing Info ---` 見出し + 箇条書き 2–3件  
        L3 = 能動提案。**must list at least two distinct actionable next steps**
  7. 出力フィールド仕様  
        • **ai_receipt**: 感情ミラーリング。<br>
           hasIssues=true→40‑120 字／false→30‑80 字。<br>
        • **detailed_analysis**: 400‑600 字（JP）／250‑350 chars（EN）、<br>
           【現状の影響】→【より良い結果に繋がる視点】の2段構造。<br>
           専門用語・命令形禁止、平易な比喩歓迎。<br>
        • **improvement_points**: 50‑200 字（JP）／30‑120 chars（EN）<br>
           単文・ベネフィット型「〜することで、〜が期待できます」。箇条書き禁止。<br>
        • **suggestion**: 完成文。L2 は Banner + 2–3 bullets、L3 は ≥2案。<br>
        • **reasoning**: ≤50 chars + “ToneBand:X”。専門語可。
  8. Mention Handling: @mention の対象変更禁止（敬称追加は可）。
  9. Language: <lang> 準拠。EN は Plain Business English。
 10. JSON Only: 出力は <format> オブジェクトのみ。キーはダブルクォーテーション。
</priority_rules>

<!-- ---------------------------------------------------------------
  LAYER 2 : ANALYSIS ENGINE
---------------------------------------------------------------- -->
<analysis_engine>
  <analysis_steps>
    1. Parse input & thread context; detect mentions.
    2. Determine functional goal (thank, request, refuse, etc.).
    3. Classify issue_pattern (Emotional→Cognitive→Actional) 優先。
    4. Map to CoreActionTag (internal hint).
    5. Judge ToneBand.
    6. Select intervention level (L1‑L3).
    7. Generate explanation fields:  
         • ai_receipt per rule 7  
         • **detailed_analysis 2‑段構造**  
         • improvement_points = one‑sentence benefit
    8. Generate suggestion  
         • L1 — tone softening per ToneBand  
         • L2 — Co‑Writing banner + 2–3 bullets, placeholders  
         • L3 — ≥2 tactics from action_playbook
    9. Compose reasoning (≤50 chars incl. ToneBand).
   10. Validate: Non‑Falsification & JSON integrity.
  </analysis_steps>

  <action_playbook>
    RP‑S1: add soft thanks
    RP‑S2: replace blame wording → neutral fact
    SH‑Q1: clarifying question
    SH‑Q2: offer quick 5‑min sync
    CH‑A1: propose small experiment / pilot
    CH‑A2: compare pros‑cons, invite feedback
    NV‑N1: welcome new idea, ask detail
    NV‑N2: build on difference → “yes‑and”
  </action_playbook>
</analysis_engine>

<!-- ---------------------------------------------------------------
  LAYER 3 : APPENDIX
---------------------------------------------------------------- -->
<appendix>

  <appendix_tag_defs>
    [Emotional]  "Impolite", "HarshTone", "MissingAcknowledgment"
    [Cognitive]  "VagueIntent", "MissingContext"
    [Actional]   "UnansweredQuestion", "UnansweredDecision", "MissingFollowUp"
  </appendix_tag_defs>

  <issue_action_mapping>
    HarshTone / Impolite              → Respect   <!-- high priority -->
    MissingAcknowledgment             → Help & Respect
    VagueIntent / MissingContext      → SpeakUp or Help
    UnansweredQuestion / Decision     → SpeakUp or Challenge  <!-- choose Challenge if decision delays project -->
    MissingFollowUp                   → Help
  </issue_action_mapping>

  <!-- Few‑shot: 新構造例 -->
  <examples>
    <!-- L1 JP Borderline Soft only -->
    <example>
      <ctx></ctx>
      <draft>取り急ぎ、これ直してくれる？</draft>
      <tags>{"lang":"japanese"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["Impolite"],
        "ai_receipt": "急ぎでお願いしたいお気持ち、伝わってきます。",
        "detailed_analysis": "【現状の影響】\n依頼の急ぎ具合は伝わる一方で、命令調に聞こえやすく、相手が負担感を抱く恐れがあります。\n\n【より良い結果に繋がる視点】\n冒頭に軽い感謝や依頼語尾を添えることで、協力したい気持ちを高めつつ、修正スピードを落とさずに済みます。",
        "improvement_points": "感謝を添えることで、同僚の協力度と対応スピードが上がります。",
        "suggestion": "お手数ですが、これ直してもらえると助かります！よろしく！",
        "reasoning": "Respect ToneBand:Borderline"
      }</output>
    </example>

    <!-- L2 EN Co‑Writing example -->
    <example>
      <ctx></ctx>
      <draft>Can you check that thing we discussed?</draft>
      <tags>{"lang":"english"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["MissingContext"],
        "ai_receipt": "Following up can be tricky without clear details.",
        "detailed_analysis": "【Current impact】\nThe vague phrase “that thing” forces your colleague to guess, delaying action.\n\n【Perspective for better results】\nBy naming the file and clarifying the deadline, you reduce their cognitive load and get feedback faster.",
        "improvement_points": "Clarifying file name and deadline speeds up feedback and prevents confusion.",
        "suggestion": "Could you please review the proposal we discussed yesterday?\n\n--- Missing Info ---\n• (1) File name: [■■■■]\n• (2) Feedback focus: [■■■■]\n• (3) Deadline: [■■■■]\n---------------------\nThanks a lot!",
        "reasoning": "SpeakUp ToneBand:Acceptable"
      }</output>
    </example>

    <!-- L3 JP 二案提案 (再掲) -->
    <example>
      <ctx>[09:00] 上司: 明日A社訪問、一緒に来られる？</ctx>
      <draft>すみません、予定があって無理です。</draft>
      <tags>{"lang":"japanese"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["MissingAcknowledgment","UnansweredDecision"],
        "ai_receipt": "参加できず申し訳ないお気持ちと、代替案を探しておられるのですね。",
        "detailed_analysis": "【現状の影響】\n断るだけでは上司が代替手段を探す追加作業を抱えます。\n\n【より良い結果に繋がる視点】\n代替同行者の提案やアフターフォローを示すことで、上司の負荷を下げつつ信頼を維持できます。",
        "improvement_points": "代替案を示すことで、上司の負担が減り信頼感が高まります。",
        "suggestion": "ご依頼ありがとうございます。あいにく外せない予定があり、明日は同行できません。\n1. 代わりにBさんへ同行をお願いしてみてはいかがでしょうか。\n2. 訪問後に議事録をご共有いただければ、フォローアップ資料を私が作成します。",
        "reasoning": "Help&Respect ToneBand:Acceptable"
      }</output>
    </example>
  </examples>

  <format>{
    "originalText": "",
    "hasIssues": false,
    "issue_pattern": [],
    "detected_mentions": [],
    "ai_receipt": "",
    "detailed_analysis": "",
    "improvement_points": "",
    "suggestion": "",
    "reasoning": ""
  }</format>
</appendix>
</system>

"""


In [39]:

# -------------------------------------------------------------------
# 元のテストケース (Original Test Cases, 1-6)
# -------------------------------------------------------------------
original_test_cases = [
    {
        "case_name": "Test Case 1: The \"Do Nothing\" Test",
        "payload": {
            "user_draft": "承知いたしました。ご依頼の件、本日17時までに対応いたします。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "承知いたしました。ご依頼の件、本日17時までに対応いたします。", "reasoning": "No issues found." }
    },
    {
        "case_name": "Test Case 2: The \"Simple Fix\" Test",
        "payload": {
            "user_draft": "明日までにこの資料、修正しといて。",
            "thread_context": "", "hierarchy": "junior", "social_distance": "somewhat_close", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["Impolite", "HarshTone"], "suggestion": "お疲れ様です。\n\nお忙しいところ恐れ入りますが、この資料を明日までに修正していただけますでしょうか？\n\nご対応よろしくお願いいたします。", "reasoning": "Impolite tone imposes emotional cost." }
    },
    {
        "case_name": "Test Case 3: The \"Over-Correction\" Test",
        "payload": {
            "user_draft": "先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "先日のA社との会議ですが、議事録を関係者に共有しておいていただけますか。", "reasoning": "Sufficiently clear for many contexts. No intervention needed." }
    },
    {
        "case_name": "Test Case 4: The \"Multiple Missing Info\" Test",
        "payload": {
            "user_draft": "例のプロジェクトの件、例の資料を更新して、展開してください。",
            "thread_context": "", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["MissingContext", "VagueIntent"], "suggestion": "お疲れ様です。プロジェクトの件でご依頼です。\n\nお手数ですが、資料を更新し、関係者の皆様へ展開をお願いいたします。\n\nメッセージを送信される前に、以下の点をご確認・ご追記いただけますでしょうか。\n\n--- Missing Info ---\n• どのプロジェクトか: [■■■■]\n• どの資料を更新するか: [■■■■]\n• 誰に展開するか: [■■■■]\n• いつまでに対応してほしいか: [■■■■]\n---------------------\n\nご確認のほど、よろしくお願いいたします。", "reasoning": "Multiple ambiguous references." }
    },
    {
        "case_name": "Test Case 5: The \"Proactive Action\" Test",
        "payload": {
            "user_draft": "すみません、明日の15時はクライアントとの最重要プレゼンと完全に重複しており、対応不可能です。",
            "thread_context": "[10:00] 上司: @ユーザーさん、急で申し訳ないんだけど、明日の15時から入っているB社との定例、代わりにファシリテーターお願いできないかな？", "hierarchy": "senior", "social_distance": "close", "language": "japanese"
        },
        "ideal_output": { "hasIssues": True, "issue_pattern": ["MissingAcknowledgment"], "suggestion": "@上司さん\n\nお声がけいただきありがとうございます！急なご用件とのことで、ぜひお力になりたかったのですが、大変申し訳ありません。明日の15時は、B社との最重要プレゼンと時間が完全に重なっておりまして…。\n\nもしよろしければ、代替案として、\n(1) 私からCさんに代理をお願いしてみる\n(2) 会議の重要な論点だけでも、事前に私にインプットいただく\nといった方法が考えられますが、いかがでしょうか？", "reasoning": "Passive rejection; needs proactive alternatives." }
    },
    {
        "case_name": "Test Case 6: The \"Context is Key\" Test",
        "payload": {
            "user_draft": "はい、その件、問題ありません。A案で進めてください。",
            "thread_context": "[14:00] 田中さん: C社の新機能開発プロジェクトの件ですが、現状のA案で進めて問題ないでしょうか？ご確認をお願いします。", "hierarchy": "peer", "social_distance": "neutral", "language": "japanese"
        },
        "ideal_output": { "hasIssues": False, "issue_pattern": [], "suggestion": "はい、その件、問題ありません。A案で進めてください。", "reasoning": "Context resolves ambiguity." }
    }
]

# -------------------------------------------------------------------
# 新しいテストケース (New Test Cases, 7-9)
# -------------------------------------------------------------------
new_test_cases = [
    {
        "case_name": "New Test Case 7: 「Polite but Harsh」の分離テスト",
        "payload": {
            "user_draft": "ご指摘ありがとうございます。ただ、そのデータは先週〇〇さんから正式に共有いただいた最新版のはずですが、何か勘違いされているのではないでしょうか。",
            "thread_context": "[11:00] 同僚: 先日のA社向け提案資料、拝見しました。素晴らしい出来ですね！ただ1点、5ページの市場規模のグラフですが、参照しているデータソースが少し古いかもしれません。",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone"],
            "suggestion": "ご指摘ありがとうございます。私が参照しているデータと認識が異なるようですので、お手数ですが、参照元のデータについて一度すり合わせさせていただけますでしょうか？",
            "reasoning": "Polite form but harsh/accusatory content."
        }
    },
    {
        "case_name": "New Test Case 8: Co-Writingモデルの応用力テスト（日程調整）",
        "payload": {
            "user_draft": "来週あたり、打ち合わせしませんか？",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "somewhat_close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "suggestion": "ぜひ打ち合わせしましょう！\n\n日程調整のため、お手数ですが以下の点について教えていただけますでしょうか？\n\n--- Missing Info ---\n• (1) 会議の目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) ご希望の候補日時（複数）: [■■■■]\n---------------------\n\nご連絡お待ちしております！",
            "reasoning": "Missing key details for scheduling."
        }
    },
    {
        "case_name": "New Test Case 9: 能動的提案の応用力テスト（専門的な依頼の断り）",
        "payload": {
            "user_draft": "ご依頼ありがとうございます。しかし、この改善にはアーキテクチャの根本的な見直しが必要で、明日までというのは物理的に不可能です。申し訳ありません。",
            "thread_context": "[16:00] 上司: @ユーザーさん、このPythonスクリプト、明日までにパフォーマンスを2倍に改善しておいてくれないか。",
            "hierarchy": "senior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingAcknowledgment"],
            "suggestion": "ご依頼ありがとうございます。パフォーマンス改善の件、承知いたしました。\n\n拝見したところ、ご期待の成果を出すにはアーキテクチャの根本的な見直しが必要となり、明日までの完了は難しい状況です。大変申し訳ありません。\n\nつきましては、まずは(1)現状のボトルネックを特定し、改善インパクトの大きい箇所から着手する、(2)改善の目標値と、そのために必要な工数についてご相談する、といった形で進めさせていただくのはいかがでしょうか。",
            "reasoning": "Passive rejection; needs proactive next steps."
        }
    }
]

# -------------------------------------------------------------------
# 新しいテストケース (New Test Cases, 10-26)
# -------------------------------------------------------------------
test_cases_10to26 = [
# ------------------------------------------------------------------
{ "case_name": "Case 10 : OK‑Polite (JP)",                   # L0
  "payload": {
     "user_draft": "おはようございます。問題ありません、10時でお願いします！",
     "thread_context": "[09:00] チーム: 週次定例は水曜 10:00 で良いですか？",
     "hierarchy": "peer", "social_distance": "neutral", "language": "japanese" },
  "ideal_output": {
     "hasIssues": False,
     "issue_pattern": [],
     "ai_receipt": "明快で前向きなご返信ですね！スムーズな調整に大いに貢献しています。",
     "detailed_analysis": "【現状の影響】\n挨拶で良好な雰囲気を作りつつ、希望時間を肯定形で即答しているため、相手は追加確認の負担なく予定を確定できます。\n\n【より良い結果に繋がる視点】\n肯定のあとに依頼語尾「お願いします」を添えていることで、依頼と感謝が同時に伝わり、今後も円滑なコミュニケーションが続く土台が築かれます。",
     "improvement_points": "肯定表現と依頼語尾の組合せで、相手の安心感と調整速度を高められます。",
     "suggestion": "おはようございます。問題ありません、10時でお願いします！",
     "reasoning": "Acceptable ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 11 : Harsh Tone 修正 (JP‑L1)",         # L1
  "payload": {
     "user_draft": "至急！この資料ミスだらけ。早く直して。",
     "thread_context": "", "hierarchy": "senior", "social_distance": "close",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["HarshTone","Impolite"],
     "ai_receipt": "資料の品質を高めたいお気持ちと、時間的な切迫感を受け取りました。",
     "detailed_analysis": "【現状の影響】\n「ミスだらけ」「早く直して」といった断定的な言葉は、受け手に『責められている』という感情を生み、迅速な相談や報告をためらわせる原因になります。\n\n【より良い結果に繋がる視点】\n冒頭で相手の状況に配慮し、誤りを具体的に示すことで、修正ポイントを正確に把握してもらえます。感謝や依頼形を添えると心理的ハードルが下がり、修正速度とモチベーションが向上します。",
     "improvement_points": "具体箇所を示し感謝を添えることで、修正スピードと協力度が上がります。",
     "suggestion": "お忙しいところ恐れ入ります。次の点をご確認いただけますか？\n1. 図表の数値誤記\n2. スライド5のタイトル表記\n至急のお願いで恐縮ですが、よろしくお願いいたします。",
     "reasoning": "Respect ToneBand:Harmful"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 12 : 情報欠落 (JP‑L2)",                # L2
  "payload": {
     "user_draft": "来週あたり打ち合わせできますか？",
     "thread_context": "", "hierarchy": "peer", "social_distance": "distant",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["VagueIntent","MissingContext"],
     "ai_receipt": "打ち合わせのご提案ありがとうございます。前向きに調整されたい意図を受け取りました。",
     "detailed_analysis": "【現状の影響】\n『来週あたり』という曖昧な表現では、相手が日程を検討するために追加質問をしなければならず、調整が遅れるリスクがあります。\n\n【より良い結果に繋がる視点】\n目的・参加者・複数候補日を示すことで、相手は即座に可否判断でき、往復回数が減ります。初対面に近い関係では、丁寧な挨拶を添えることで信頼構築も進みます。",
     "improvement_points": "目的・参加者・候補日を明示すれば日程調整の往復回数を減らせます。",
     "suggestion": "はじめまして、△△部の□□です。\n打ち合わせを検討しております。\n\n--- Missing Info ---\n• (1) 目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) ご希望の候補日時（複数）: [■■■■]\n---------------------\nご確認のほどお願いいたします。",
     "reasoning": "SpeakUp ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 13 : 軽度の無礼 (JP‑Borderline)",     # L1
  "payload": {
     "user_draft": "資料送っといて。",
     "thread_context": "", "hierarchy": "peer", "social_distance": "close",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["Impolite"],
     "ai_receipt": "資料共有をお願いしたいお気持ち、理解しました。",
     "detailed_analysis": "【現状の影響】\n命令形の一言だけでは、相手が『急かされている』と感じやすく、返信遅延やモチベーション低下につながる可能性があります。\n\n【より良い結果に繋がる視点】\n軽い挨拶と感謝を添えるだけで、同僚は協力的な気持ちになり、資料提出がスムーズに進みます。",
     "improvement_points": "感謝を添えることで、同僚の協力意欲と提出速度が高まります。",
     "suggestion": "お手数ですが資料をご共有いただけると助かります！ありがとう！",
     "reasoning": "Respect ToneBand:Borderline"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 14 : MissingAcknowledgment (JP‑L1)",  # L1
  "payload": {
     "user_draft": "@Ken 前回の報告書まだ？",
     "thread_context": "[火] Ken: 報告書ドラフト完成しました。",
     "hierarchy": "senior", "social_distance": "close",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["MissingAcknowledgment"],
     "ai_receipt": "状況確認を急ぎたいお気持ち、理解しました。",
     "detailed_analysis": "【現状の影響】\nKen さんの成果を無視した催促は、『努力が認められていない』という感情を招き、報告ペースが遅くなる恐れがあります。\n\n【より良い結果に繋がる視点】\n先に感謝を伝えたうえで進捗を尋ねると、相手は承認を感じ、情報共有を積極的に行いやすくなります。",
     "improvement_points": "感謝を示してから進捗確認すると、協力度が上がります。",
     "suggestion": "Ken さん、ドラフト作成ありがとうございます！\n前回の報告書の進捗状況を教えていただけますか？",
     "reasoning": "Help&Respect ToneBand:Borderline"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 15 : MissingContext (EN‑L2)",         # L2
  "payload": {
     "user_draft": "Can you check that thing we discussed?",
     "thread_context": "", "hierarchy": "peer", "social_distance": "neutral",
     "language": "english" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["MissingContext"],
     "ai_receipt": "Following up can be tricky when details are vague.",
     "detailed_analysis": "【Current impact】\nThe phrase “that thing” forces your colleague to guess, causing delays.\n\n【Perspective for better results】\nBy naming the file and clarifying the deadline, you cut cognitive load and receive feedback sooner.",
     "improvement_points": "Naming the file and deadline accelerates feedback and avoids confusion.",
     "suggestion": "Could you please review the proposal we discussed yesterday?\n\n--- Missing Info ---\n• (1) File name: [■■■■]\n• (2) Feedback focus: [■■■■]\n• (3) Deadline: [■■■■]\n---------------------\nThanks a lot!",
     "reasoning": "SpeakUp ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 16 : UnansweredDecision (JP‑L3)",      # L3
  "payload": {
     "user_draft": "了解です。よろしく。",
     "thread_context": "[上司] 新ツール採用 A案とB案どちらにしますか？",
     "hierarchy": "junior", "social_distance": "neutral",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["UnansweredDecision"],
     "ai_receipt": "決定の意思表示に迷われているようですね。",
     "detailed_analysis": "【現状の影響】\n『了解です』だけでは選択肢が提示されたまま停滞し、上司は追加確認をしなければならず、意思決定が遅れます。\n\n【より良い結果に繋がる視点】\nA案とB案を比較し、推奨する案と理由をセットで示すことで、上司は即座に承認でき、導入作業を前倒しできます。",
     "improvement_points": "推奨案と理由を示すことで、上司が即断しやすくなります。",
     "suggestion": "A案を推奨いたします。理由は 1) 既存システムとの互換性 2) コスト面 です。\nご確認のほどお願いいたします。\nまたは、B案を採用する場合のメリットはサポート体制の充実です。ご判断いただけますと幸いです。",
     "reasoning": "Challenge ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 17 : UnansweredQ + MissingFU (JP‑L3)", # L3
  "payload": {
     "user_draft": "進めておいて。",
     "thread_context": "[同僚] 明日の提案資料ですが、フォーマットはこれで良いですか？",
     "hierarchy": "senior", "social_distance": "distant",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["UnansweredQuestion","MissingFollowUp"],
     "ai_receipt": "資料の方向性を示す必要性を感じながら、時間が迫っておられるのですね。",
     "detailed_analysis": "【現状の影響】\n質問に答えずに『進めておいて』だけでは、同僚は誤った方向で作業を進め、再修正が発生する恐れがあります。\n\n【より良い結果に繋がる視点】\nフォーマット承認→締切→自分のサポートを示す順で伝えると、相手は安心して作業に集中でき、全体の時間短縮に繋がります。",
     "improvement_points": "フォーマット承認と締切提示で、作業の手戻りを防げます。",
     "suggestion": "資料フォーマットは現行テンプレートで進めましょう。\n初稿を本日17:00までに共有いただければ、レビューを私が担当します。不明点があれば教えてください。",
     "reasoning": "Help ToneBand:Borderline"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 18 : Harsh+Vague (EN‑L2)",            # L2
  "payload": {
     "user_draft": "What were you thinking? Fix it by tomorrow.",
     "thread_context": "", "hierarchy": "senior", "social_distance": "close",
     "language": "english" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["HarshTone","VagueIntent"],
     "ai_receipt": "I sense urgency and frustration about the current quality.",
     "detailed_analysis": "【Current impact】\nThe accusatory opening may trigger defensiveness, and “it” is unclear, risking wasted effort.\n\n【Perspective for better results】\nNaming the exact slide and offering help keeps urgency while preserving collaboration.",
     "improvement_points": "Specifying the issue and adding appreciation keeps urgency without harming rapport.",
     "suggestion": "Thanks for your work so far. Could you please update slide 3’s figures and adjust the layout by tomorrow? Let me know if you need support.",
     "reasoning": "Respect ToneBand:Harmful"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 19 : Very‑distant Polite (JP‑OK)",     # L0
  "payload": {
     "user_draft": "取締役　平素よりお世話になっております。先日ご検討いただいた件につき、ご意見を賜りたく存じます。",
     "thread_context": "", "hierarchy": "junior", "social_distance": "very_distant",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": False, "issue_pattern": [],
     "ai_receipt": "取締役への丁重な依頼文、礼節が行き届いていますね。",
     "detailed_analysis": "【現状の影響】\n丁寧な敬語と簡潔な依頼内容により、目上の方でも内容をすぐ理解でき、負担感なく返信できます。\n\n【より良い結果に繋がる視点】\n冒頭で案件名を一言添えると、取締役は議題を瞬時に想起でき、回答スピードがさらに向上します。",
     "improvement_points": "案件名を冒頭に示すと、回答スピードがさらに上がります。",
     "suggestion": "取締役　平素よりお世話になっております。先日ご検討いただいた件につき、ご意見を賜りたく存じます。",
     "reasoning": "Acceptable ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 20 : Very‑close Casual OK (JP)",       # L0
  "payload": {
     "user_draft": "ヨシ！この案でゴーしよ！👍",
     "thread_context": "[チーム] 新機能ネーミング案の選定中",
     "hierarchy": "peer", "social_distance": "very_close",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": False, "issue_pattern": [],
     "ai_receipt": "勢いのある合意表現で、チームの前向きさが伝わります！",
     "detailed_analysis": "【現状の影響】\nカジュアルな合意表現と絵文字がチームの一体感を高め、決定事項を印象づけます。\n\n【より良い結果に繋がる視点】\n理由を一言添えると全員の納得感がさらに高まり、実行フェーズへの移行がスムーズになります。",
     "improvement_points": "決定理由を添えると、全員の納得感が高まります。",
     "suggestion": "ヨシ！この案でゴーしよ！👍",
     "reasoning": "Acceptable ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 21 : Borderline Soft‑Fix (JP‑L1)",     # L1
  "payload": {
     "user_draft": "取り急ぎ、これ直してくれる？",
     "thread_context": "", "hierarchy": "peer", "social_distance": "close",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["Impolite"],
     "ai_receipt": "急ぎでお願いしたいお気持ち、伝わってきます。",
     "detailed_analysis": "【現状の影響】\n命令形は相手にプレッシャーを与え、対応が後回しになることがあります。\n\n【より良い結果に繋がる視点】\n謝意を添えるだけで協力的な雰囲気になり、修正が早まります。",
     "improvement_points": "謝意を添えることで、協力度と修正スピードが上がります。",
     "suggestion": "お疲れ！これ直してもらえると助かります。よろしく！",
     "reasoning": "Respect ToneBand:Borderline"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 22 : Info Gap (JP‑L2)",                # L2
  "payload": {
     "user_draft": "来週どこかでミーティングしましょう。",
     "thread_context": "", "hierarchy": "peer", "social_distance": "distant",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["MissingContext","VagueIntent"],
     "ai_receipt": "打ち合わせをご提案いただきありがとうございます。詳細が分かると調整しやすいですね。",
     "detailed_analysis": "【現状の影響】\n『どこかで』という表現では場所も目的も不明確で、相手は何を準備すれば良いか分からず調整が遅れます。\n\n【より良い結果に繋がる視点】\n目的・参加者・複数日時を示すことで、相手は即決しやすく、会議準備も進めやすくなります。",
     "improvement_points": "目的と候補日時を示すと、調整の往復が減ります。",
     "suggestion": "お世話になっております。\n来週あたり、打ち合わせの機会をいただけますでしょうか。\n\n--- Missing Info ---\n• (1) 打ち合わせの目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) ご都合の良い日時（複数候補）: [■■■■]\n---------------------\nご検討のほどよろしくお願いいたします。",
     "reasoning": "SpeakUp ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 23 : Harsh Fix (EN‑L1)",               # L1
  "payload": {
     "user_draft": "This report is a mess. Fix it NOW.",
     "thread_context": "", "hierarchy": "senior", "social_distance": "close",
     "language": "english" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["HarshTone","Impolite"],
     "ai_receipt": "Urgency is clear, and you'd like the fix handled quickly.",
     "detailed_analysis": "【Current impact】\nThe harsh wording can hurt morale and delay honest feedback.\n\n【Perspective for better results】\nA polite request with specific guidance and thanks keeps urgency while preserving motivation.",
     "improvement_points": "Polite guidance plus thanks keeps urgency without harming rapport.",
     "suggestion": "Hi team, I found several issues in the report. Could you please revise it at your earliest convenience? Let me know if you need any support. Thank you.",
     "reasoning": "Respect ToneBand:Harmful"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 24 : L3 Alt‑Plan (JP)",                # L3
  "payload": {
     "user_draft": "すみません、明日の顧客訪問は参加できません。",
     "thread_context": "[09:00] 上司: 明日A社訪問、一緒に行ける？",
     "hierarchy": "junior", "social_distance": "neutral",
     "language": "japanese" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["MissingAcknowledgment","UnansweredDecision"],
     "ai_receipt": "参加できず心苦しいお気持ちと、代替案を模索されているのですね。",
     "detailed_analysis": "【現状の影響】\n断りだけでは訪問体制が決まらず、上司が代替手段を探す追加負荷が発生します。\n\n【より良い結果に繋がる視点】\n代替同行者提案やフォロー策を示すことで、上司の負荷を下げつつ顧客対応品質を保てます。",
     "improvement_points": "代替案を示すと、上司の負担を減らし信頼感を保てます。",
     "suggestion": "ご依頼ありがとうございます。あいにく外せない予定があり、明日の訪問には同行できません。\n1. 代わりにBさんへ同行をお願いしてみるのはいかがでしょうか。\n2. 訪問後に議事録を共有いただければ、フォローアップ資料を私が作成します。",
     "reasoning": "Help&Respect ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 25 : Help Request (EN‑Help)",          # L2→Help
  "payload": {
     "user_draft": "Could someone explain the new deployment script? I'm lost.",
     "thread_context": "", "hierarchy": "peer", "social_distance": "neutral",
     "language": "english" },
  "ideal_output": {
     "hasIssues": True, "issue_pattern": ["MissingContext"],
     "ai_receipt": "Great that you're reaching out instead of getting stuck.",
     "detailed_analysis": "【Current impact】\nWithout details, helpers may not know where to start, delaying support.\n\n【Perspective for better results】\nStating the confusing step and offering a quick call makes it easier for peers to jump in.",
     "improvement_points": "Naming the confusing step and offering a brief call speeds up assistance.",
     "suggestion": "Could anyone spare 10 minutes to walk me through the new deployment script? I'm unsure about the staging step.",
     "reasoning": "Help ToneBand:Acceptable"
}},
# ------------------------------------------------------------------
{ "case_name": "Case 26 : Novelty Welcome (EN‑OK)",         # L0 (New)
  "payload": {
     "user_draft": "I have a rather unconventional idea for our onboarding flow.",
     "thread_context": "", "hierarchy": "peer", "social_distance": "neutral",
     "language": "english" },
  "ideal_output": {
     "hasIssues": False, "issue_pattern": [],
     "ai_receipt": "Love that you're bringing a fresh perspective to the team!",
     "detailed_analysis": "【Current impact】\nIntroducing novelty signals psychological safety and can spark creative solutions.\n\n【Perspective for better results】\nAdding a quick teaser of the benefit—like reducing drop‑off—helps teammates engage with the idea more eagerly.",
     "improvement_points": "Stating the expected benefit draws teammates into the discussion.",
     "suggestion": "I have a rather unconventional idea for our onboarding flow. It could lower our first‑week drop‑off—happy to share details if you’re interested!",
     "reasoning": "Novelty ToneBand:Acceptable"
}}
]



# -------------------------------------------------------------------
# テスト実行と結果表示の関数 (Test Execution and Display Functions)
# -------------------------------------------------------------------

def run_single_test(api_url: str, payload: dict) -> dict:
    """指定されたペイロードでAPIにリクエストを送信し、結果を返す"""
    """Sends a request to the API with the given payload and returns the result."""
    # Note: The original script was not passing the system prompt.
    # This has been corrected to include the system_prompt in the payload.
    full_payload = payload.copy()
    # This assumes the API backend is adapted to receive the system prompt in the body.
    # このAPIバックエンドがボディでシステムプロンプトを受け取るように適合されていることを前提とします。
    full_payload['system_prompt'] = SYSTEM_PROMPT

    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(api_url, data=json.dumps(full_payload), headers=headers)
        response.raise_for_status()  # エラーがあれば例外を発生させる
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def create_comparison_table(case_name: str, payload: dict, ideal: dict, actual: dict) -> str:
    """比較結果のHTMLテーブルを生成する"""
    """Generates an HTML table for comparison results."""
    
    style = """<style> .comparison-table { width: 100%; border-collapse: collapse; margin-bottom: 20px; font-family: sans-serif; } .comparison-table th, .comparison-table td { border: 1px solid #ddd; padding: 8px; text-align: left; vertical-align: top; } .comparison-table th { background-color: #f2f2f2; font-weight: bold; } .comparison-table pre { white-space: pre-wrap; word-wrap: break-word; margin: 0; font-size: 13px; } .status-ok { color: green; font-weight: bold; } .status-ng { color: red; font-weight: bold; } </style>"""
    
    html = f"<h3>{case_name}</h3>"
    display_payload = {k:v for k,v in payload.items() if k != 'system_prompt'}
    html += f"<pre>Input: {json.dumps(display_payload, indent=2, ensure_ascii=False)}</pre>"
    html += "<table class='comparison-table'>"
    html += "<tr><th>評価項目（キー）</th><th>理想のアウトプット</th><th>実際のAIアウトプット</th><th>コメント</th></tr>"
    
    all_keys = sorted(list(set(ideal.keys()) | set(actual.keys())))
    
    for key in all_keys:
        ideal_val = ideal.get(key, "N/A")
        actual_val = actual.get(key, "N/A")
        
        comment = ""
        if key == 'issue_pattern' and isinstance(ideal_val, list) and isinstance(actual_val, list):
             if sorted(ideal_val) == sorted(actual_val):
                 comment = "<span class='status-ok'>✅ 一致</span>"
             else:
                 comment = "<span class='status-ng'>❌ 不一致</span>"
#        elif str(ideal_val) == str(actual_val):
#            comment = "<span class='status-ok'>✅ 一致</span>"
#        else:
#            comment = "<span class='status-ng'>❌ 不一致</span>"

        if isinstance(ideal_val, (dict, list)): ideal_val = json.dumps(ideal_val, indent=2, ensure_ascii=False)
        if isinstance(actual_val, (dict, list)): actual_val = json.dumps(actual_val, indent=2, ensure_ascii=False)
            
        html += f"""<tr><td><code>{key}</code></td><td><pre>{ideal_val}</pre></td><td><pre>{actual_val}</pre></td><td>{comment}</td></tr>"""
        
    html += "</table>"
    return style + html


In [41]:

# -------------------------------------------------------------------
# メイン処理 (Main Process)
# -------------------------------------------------------------------

# 実行したいテストケースのセットを選択してください
# Select the set of test cases you want to run
#RUN_NEW_CASES = False  # 元のテスト(1-6)を実行する場合
#RUN_NEW_CASES = True # 新しいテスト(7-9)を実行する場合

# ここで切り替え
# Switch here
test_cases_to_run = test_cases_10to26
#test_cases_to_run = new_test_cases 
#test_cases_to_run = original_test_cases

print(f"テストを開始します... APIエンドポイント: {API_URL}")
print(f"Executing tests... API Endpoint: {API_URL}")

all_results_html = ""
for test in test_cases_to_run:
    case_name = test["case_name"]
    payload = test["payload"]
    ideal_output = test["ideal_output"]
    
    print(f"\n--- {case_name} を実行中 ---")
    
    actual_output = run_single_test(API_URL, payload)
    print(actual_output)
    
    if "error" in actual_output:
        print(f"エラーが発生しました: {actual_output['error']}")
        all_results_html += f"<h3>{case_name}</h3><p style='color:red;'>テスト実行エラー: {actual_output['error']}</p>"
    else:
        comparison_keys = ['hasIssues', 'issue_pattern', 'suggestion', 'reasoning']
        ideal_subset = {k: ideal_output.get(k) for k in comparison_keys}
        actual_subset = {k: actual_output.get(k) for k in comparison_keys}
        
        table_html = create_comparison_table(case_name, payload, ideal_subset, actual_subset)
        all_results_html += table_html

print("\n--- 全てのテストが完了しました ---")

display(HTML(all_results_html))


テストを開始します... APIエンドポイント: http://localhost:3000/api/check-tone
Executing tests... API Endpoint: http://localhost:3000/api/check-tone

--- Case 10 : OK‑Polite (JP) を実行中 ---
{'originalText': 'おはようございます。問題ありません、10時でお願いします！', 'hasIssues': False, 'issue_pattern': [], 'detected_mentions': [], 'ai_receipt': '丁寧な挨拶と了承の表現で、スムーズな合意形成が感じられます。', 'detailed_analysis': '【現状の影響】\n挨拶から始まり、明確に了承を伝えているため、相手に安心感を与え、コミュニケーションが円滑に進みます。\n\n【より良い結果に繋がる視点】\nこの調子で、必要に応じて具体的な準備事項や確認事項も共有すると、さらに効率的な定例会議運営が期待できます。', 'improvement_points': '丁寧な表現を続けることで、チーム内の信頼関係と円滑な連携が促進されます。', 'suggestion': 'おはようございます。問題ありません、10時でお願いします！', 'reasoning': 'ToneBand:Acceptable'}

--- Case 11 : Harsh Tone 修正 (JP‑L1) を実行中 ---
{'originalText': '至急！この資料ミスだらけ。早く直して。', 'hasIssues': True, 'issue_pattern': ['HarshTone', 'Impolite'], 'detected_mentions': [], 'ai_receipt': '急いで対応してほしい気持ちが伝わりますが、強い口調で相手に負担を感じさせています。', 'detailed_analysis': '【現状の影響】\n強い命令調での依頼は、受け手にプレッシャーやストレスを与え、モチベーション低下や対応の遅れにつながる恐れがあります。\n\n【より良い結果に繋がる視点】\n感謝やお願いの言葉を添え、柔らかい表現にす

評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
hasIssues,False,False,
issue_pattern,[],[],✅ 一致
reasoning,Acceptable ToneBand:Acceptable,ToneBand:Acceptable,
suggestion,おはようございます。問題ありません、10時でお願いします！,おはようございます。問題ありません、10時でお願いします！,
評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
hasIssues,True,True,
issue_pattern,"[ ""HarshTone"", ""Impolite"" ]","[ ""HarshTone"", ""Impolite"" ]",✅ 一致
reasoning,Respect ToneBand:Harmful,Respect ToneBand:Harmful,
suggestion,お忙しいところ恐れ入ります。次の点をご確認いただけますか？ 1. 図表の数値誤記 2. スライド5のタイトル表記 至急のお願いで恐縮ですが、よろしくお願いいたします。,お忙しいところ恐れ入りますが、この資料にいくつか誤りがあるようです。お手すきの際にご確認いただき、修正をお願いできますでしょうか。よろしくお願いいたします。,
評価項目（キー）,理想のアウトプット,実際のAIアウトプット,コメント
